In [1]:
import thunder as td
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
%matplotlib inline
from regression import LinearRegression
import tifffile

In [2]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# conf = (SparkConf()
#          .setMaster("local")
#          .setAppName("My app")
#          .set("spark.executor.memory", "40g")
#          .set("spark.ui.port","8080")
#          .set("spark.driver.host","127.0.0.1")
#          .set("spark.local.dir","/mnt/tmp/")
#        )

# sc = SparkContext(conf = conf)
sc = SparkContext()

In [ ]:
sc.stop()

In [3]:
#input number of planes
filename='/mnt/downloads/Itia/Fish201712144_ELO_IRO_HS.tif'
data = tifffile.imread(filename)
tifffile.imsave(filename[:-4]+'_mean.tif',data.mean(axis=0).astype(np.uint16))
data = td.images.fromarray(data, engine=sc, npartitions=5000)
data = data.repartition(5000)
data = data[20:150,:,:,:]

In [4]:
from registration import CrossCorr
algorithm = CrossCorr()
reference=np.load('/mnt/downloads/Itia/reference.npy')
model = algorithm.fit(data,reference)
data=model.transform(data)
data.max_projection(axis=0).totif(filename[:-4]+'_warp.tif')

In [5]:
data = data.toseries().detrend('linear').normalize()
data=data.repartition(5000)
data

Series
mode: spark
dtype: float64
shape: (25, 540, 640, 130)

In [6]:
Stimuli=np.zeros(130)
GCaMP6=[0,1.69644104899772,5.13796058542217,8.27886020152244,10.3756715204800,11.8173714529814,12.2425184714093,10.8571417354877,8.80831829681196,6.91339112244670,5.46959264663869,4.30868766622567,3.42533619066766,2.75378443486879,2.18017250852183,1.72816235135824,1.32732537295463,1.00684435500268,0.730210038304555,0.530242444093118,0.362253250339685,0.227668255288566,0.0869242416152502,0.000718266708050853,-0.0828334873368325]
idxStart=20
for i in range(1,4):    
    Stimuli[(idxStart+(i-1)*40):(idxStart+(i-1)*40)+len(GCaMP6)]=GCaMP6;
Stimuli=Stimuli.reshape((1,130))
from regression import LinearRegression
algorithm = LinearRegression()
model = algorithm.fit(Stimuli.T,data)
r2 = model.score(Stimuli.T,data)
r2=r2.toarray()
betas = model.betas.toarray()
scipy.io.savemat(filename[:-4]+'.mat',mdict={'Betas':betas,'rsquared':r2})

In [7]:
sc.stop()

In [ ]:
r2_d

In [ ]:
np.save('/mnt/downloads/Itia/reference.npy',reference)

In [ ]:
reference=tifffile.imread('/mnt/downloads/Itia/Fish201711281_ERO_HS.tif')
reference=reference.mean(axis=0)